In [70]:

import numpy as np
import cv2
import pandas as pd
from scipy.stats import skew, kurtosis, entropy
from skimage.feature import local_binary_pattern, graycomatrix, graycoprops
from skimage.measure import label, regionprops
from scipy.ndimage import convolve
from math import sqrt
import os
from tqdm import tqdm
import mahotas


In [71]:
def lbp_feature(image):
    if len(image.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    i_max = np.max(image)
    i_min = np.min(image)
    if i_max - i_min != 0:
        image = (image - i_min) / (i_max - i_min)
    lbp = local_binary_pattern(image, R=1, P=8, method="uniform")  
    hist, bins = np.histogram(lbp.ravel(), bins=10, range=(0, 10))
    return {f"LBP_{i}": hist[i] for i in range(len(hist))}

In [72]:
def texture_feature(image):
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist = cv2.calcHist([gray_img],[0],None, [256], [0,256])
    hist = hist/hist.sum()

    intensities = np.arange(256)
    mean = np.sum(intensities * hist.flatten())
    std = np.sqrt(np.sum((intensities-mean)**2 *hist.flatten()))
    uniformity = np.sum(hist.flatten()**2)
    third_moment = np.sum((intensities-mean)**3 * hist.flatten())
    return {
        "texture_mean": mean,
        "texture_std": std,
        "texture_uniformity": uniformity,
        "texture_third_moment": third_moment
    }

In [73]:
def color_feature(image):
    ##rgb
    b, g, r = cv2.split(image)
    mean_r, mean_g, mean_b = np.mean(r), np.mean(g), np.mean(b)
    rs, gs, bs = sqrt(mean_r), np.sqrt(mean_g), np.sqrt(mean_b)
    return {"mean_r": mean_r,
        "mean_g": mean_g,
        "mean_b": mean_b,
        "red_sqr": rs,
        "green_sqr": gs,
        "blue_sqr": bs,
    }

In [74]:
def enhanced_color_feature(image):

    b, g, r = cv2.split(image)
    mean_r, mean_g, mean_b = np.mean(r), np.mean(g), np.mean(b)
    std_r, std_g, std_b = np.std(r), np.std(g), np.std(b)
    rs, gs, bs = np.sqrt(mean_r), np.sqrt(mean_g), np.sqrt(mean_b) 
    skew_r, skew_g, skew_b = skew(r.ravel()), skew(g.ravel()), skew(b.ravel())
    kurtosis_r, kurtosis_g, kurtosis_b = kurtosis(r.ravel()), kurtosis(g.ravel()), kurtosis(b.ravel())

    ## HSV 
    hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_img)
    mean_h, mean_s, mean_v = np.mean(h), np.mean(s), np.mean(v)
    std_h, std_s, std_v = np.std(h), np.std(s), np.std(v)
    hs, ss, vs = np.sqrt(mean_h), np.sqrt(mean_s), np.sqrt(mean_v)
    skew_h, skew_s, skew_v = skew(h.ravel()), skew(s.ravel()), skew(v.ravel())
    kurtosis_h, kurtosis_s, kurtosis_v = kurtosis(h.ravel()), kurtosis(s.ravel()), kurtosis(v.ravel())
    
    ## Ycrbr
    ycrcb_img = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(ycrcb_img)
    mean_y, mean_cr, mean_cb = np.mean(y), np.mean(cr), np.mean(cb)
    std_y, std_cr, std_cb = np.std(y), np.std(cr), np.std(cb)
    ys, crs, cbs = np.sqrt(mean_y), np.sqrt(mean_cr), np.sqrt(mean_cb)
    skew_y, skew_cr, skew_cb = skew(y.ravel()), skew(cr.ravel()), skew(cb.ravel())
    kurtosis_y, kurtosis_cr, kurtosis_cb = kurtosis(y.ravel()), kurtosis(cr.ravel()), kurtosis(cb.ravel())

    ## Lab
    lab_img = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)
    l, a, b_lab = cv2.split(lab_img)
    mean_l, mean_a, mean_b_lab = np.mean(l), np.mean(a), np.mean(b_lab)
    std_l, std_a, std_b_lab = np.std(l), np.std(a), np.std(b_lab)
    ls, as_, bs_lab = np.sqrt(mean_l), np.sqrt(mean_a), np.sqrt(mean_b_lab)
    skew_l, skew_a, skew_b_lab = skew(l.ravel()), skew(a.ravel()), skew(b_lab.ravel())
    kurtosis_l, kurtosis_a, kurtosis_b_lab = kurtosis(l.ravel()), kurtosis(a.ravel()), kurtosis(b_lab.ravel())
    
    return {
        "mean_r": mean_r, "mean_g": mean_g, "mean_b": mean_b,
        "std_r": std_r, "std_g": std_g, "std_b": std_b,
        "red_sqr": rs, "green_sqr": gs, "blue_sqr": bs,
        "skew_r": skew_r, "skew_g": skew_g, "skew_b": skew_b,
        "kurtosis_r": kurtosis_r, "kurtosis_g": kurtosis_g, "kurtosis_b": kurtosis_b,

        "mean_h": mean_h, "mean_s": mean_s, "mean_v": mean_v,
        "std_h": std_h, "std_s": std_s, "std_v": std_v,
        "hue_sqr": hs, "sat_sqr": ss, "val_sqr": vs,
        "skew_h": skew_h, "skew_s": skew_s, "skew_v": skew_v,
        "kurtosis_h": kurtosis_h, "kurtosis_s": kurtosis_s, "kurtosis_v": kurtosis_v,
        
        "mean_y": mean_y, "mean_cr": mean_cr, "mean_cb": mean_cb,
        "std_y": std_y, "std_cr": std_cr, "std_cb": std_cb,
        "y_sqr": ys, "cr_sqr": crs, "cb_sqr": cbs,
        "skew_y": skew_y, "skew_cr": skew_cr, "skew_cb": skew_cb,
        "kurtosis_y": kurtosis_y, "kurtosis_cr": kurtosis_cr, "kurtosis_cb": kurtosis_cb,

        "mean_l": mean_l, "mean_a": mean_a, "mean_b_lab": mean_b_lab,
        "std_l": std_l, "std_a": std_a, "std_b_lab": std_b_lab,
        "l_sqr": ls, "a_sqr": as_, "b_lab_sqr": bs_lab,
        "skew_l": skew_l, "skew_a": skew_a, "skew_b_lab": skew_b_lab,
        "kurtosis_l": kurtosis_l, "kurtosis_a": kurtosis_a, "kurtosis_b_lab": kurtosis_b_lab
    }


In [75]:
def zernike_feature(image, degree=8):
    if len(image.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
    blur = cv2.GaussianBlur(image, (3, 3), 0)
    threshold, new_img = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(new_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)
    
    mask = np.zeros(new_img.shape, dtype=np.uint8)
    cv2.drawContours(mask, [contour], -1, 255, -1)  
    
    # Convert mask to binary for mahotas
    binary_mask = (mask > 0).astype(np.uint8)
    

    (x_center, y_center), radius = cv2.minEnclosingCircle(contour)

    radius = int(np.ceil(radius))
    

    x_center = int(x_center)
    y_center = int(y_center)
    x1 = max(x_center - radius, 0)
    y1 = max(y_center - radius, 0)
    x2 = x_center + radius
    y2 = y_center + radius
    
    cropped_mask = binary_mask[y1:y2, x1:x2]
    
    # padding
    h, w = cropped_mask.shape
    if h != w:
        size = max(h, w)
        square_mask = np.zeros((size, size), dtype=np.uint8)
        y_offset = (size - h) // 2
        x_offset = (size - w) // 2
        square_mask[y_offset:y_offset+h, x_offset:x_offset+w] = cropped_mask
    else:
        square_mask = cropped_mask

    effective_radius = square_mask.shape[0] // 2
    zernike_moments = mahotas.features.zernike_moments(square_mask, effective_radius, degree)
    
    return {f'zernike_{i}': zernike_moments[i] for i in range(len(zernike_moments))}

In [76]:
def color_structure_descriptor(image, grid_size=(8, 8), bins=32):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    
    height, width = hsv_image.shape[:2]
    cell_h, cell_w = height // grid_size[0], width // grid_size[1]

    
    hist_bins = [bins] 

    
    csd = []

    for row in range(grid_size[0]):
        for col in range(grid_size[1]):
            x_start, y_start = col * cell_w, row * cell_h
            x_end, y_end = x_start + cell_w, y_start + cell_h
            cell = hsv_image[y_start:y_end, x_start:x_end]

            hist = cv2.calcHist([cell], [0], None, hist_bins, [0, 180])
            hist = hist.flatten() 
            hist = hist / hist.sum() if hist.sum() != 0 else hist

            csd.extend(hist)

    return {f"csd_{i}":csd[i] for i in range(len(csd))}


In [77]:
def hu_moments(image):
    if len(image.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(image, (3, 3), 0)
    threshold, new_img = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(new_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)
    moments = cv2.moments(contour)
    hu_moments = cv2.HuMoments(moments)
    hu_moments = -np.sign(hu_moments) * np.log10(np.abs(hu_moments) + 1e-10)
    hu_features = {
    "hu_1": float(hu_moments[0][0]),
    "hu_2": float(hu_moments[1][0]),
    "hu_3": float(hu_moments[2][0]),
    "hu_4": float(hu_moments[3][0]),
    "hu_5": float(hu_moments[4][0]),
    "hu_6": float(hu_moments[5][0]),
    "hu_7": float(hu_moments[6][0]),
    }

    return hu_features


In [78]:
def basic_feature(image):
    if len(image.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(image, (3, 3), 0)
    threshold, new_img = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    # new_img = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    #                            cv2.THRESH_BINARY_INV, 11, 2)
    area = np.count_nonzero(new_img)
    contours, _ = cv2.findContours(new_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(contour)
    
    length = x + w
    width = y + h
    ratio = length / width
    
    ellipse = cv2.fitEllipse(contour)
    
    major_axis_length = max(ellipse[1])
    minor_axis_length = min(ellipse[1])

    hull = cv2.convexHull(contour)
    hull_area = cv2.contourArea(hull)
    hull_perimeter = cv2.arcLength(hull, True)
    return {
        "area": area,
        "length": length,
        "width": width,
        "ratio": ratio,
        "major_axis_length": major_axis_length,
        "minor_axis_length": minor_axis_length,
        "convex_hull_area": hull_area,
        "convex_hull_perimeter": hull_perimeter,
    }

In [79]:
from scipy.ndimage import convolve
def extract_edge_histogram_features(image):

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    kernels = {
        "vertical": np.array([[-1,  2, -1],
                              [-1,  2, -1],
                              [-1,  2, -1]], dtype=np.float32),
        "horizontal": np.array([[-1, -1, -1],
                                [ 2,  2,  2],
                                [-1, -1, -1]], dtype=np.float32),
        "diag_45": np.array([[-1, -1,  2],
                             [-1,  2, -1],
                             [ 2, -1, -1]], dtype=np.float32),
        "diag_135": np.array([[ 2, -1, -1],
                              [-1,  2, -1],
                              [-1, -1,  2]], dtype=np.float32),
        "non_directional": np.array([[1,  1,  1],
                                     [1, -8,  1],
                                     [1,  1,  1]], dtype=np.float32)
    }
    
    features = {}
    total_energy = 0.0
    
    for key, kernel in kernels.items():
        response = convolve(gray.astype(np.float32), kernel, mode="reflect")
        energy = np.sum(np.abs(response))
        features[f"edge_energy_{key}"] = energy
        total_energy += energy
    
    if total_energy > 0:
        for key in list(features.keys()):
            features[key] /= total_energy

    return features

In [88]:
def extract_edge_histogram_features_2(image):
    # Convert to grayscale if necessary
    if len(image.shape) > 2:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()
    
    # Define convolution kernels for a variety of edge types.
    kernels = {
        # "vertical": np.array([[-1,  2, -1],
        #                       [-1,  2, -1],
        #                       [-1,  2, -1]], dtype=np.float32),
        # "horizontal": np.array([[-1, -1, -1],
        #                         [ 2,  2,  2],
        #                         [-1, -1, -1]], dtype=np.float32),
        # "diag_45": np.array([[-1, -1,  2],
        #                      [-1,  2, -1],
        #                      [ 2, -1, -1]], dtype=np.float32),
        # "diag_135": np.array([[ 2, -1, -1],
        #                       [-1,  2, -1],
        #                       [-1, -1,  2]], dtype=np.float32),
        # "non_directional": np.array([[1,  1,  1],
        #                              [1, -8,  1],
        #                              [1,  1,  1]], dtype=np.float32),
        # Sobel Operators
        "sobel_x": np.array([[-1, 0, 1],
                             [-2, 0, 2],
                             [-1, 0, 1]], dtype=np.float32),
        "sobel_y": np.array([[-1, -2, -1],
                             [ 0,  0,  0],
                             [ 1,  2,  1]], dtype=np.float32),
        # Prewitt Operators
        "prewitt_x": np.array([[-1, 0, 1],
                               [-1, 0, 1],
                               [-1, 0, 1]], dtype=np.float32),
        "prewitt_y": np.array([[-1, -1, -1],
                               [ 0,  0,  0],
                               [ 1,  1,  1]], dtype=np.float32),
        # Laplacian Operators
        "laplacian_4": np.array([[0,  1,  0],
                                 [1, -4,  1],
                                 [0,  1,  0]], dtype=np.float32),
        "laplacian_8": np.array([[1,  1,  1],
                                 [1, -8,  1],
                                 [1,  1,  1]], dtype=np.float32),
        # Roberts Cross Operators
        "roberts_x": np.array([[1, 0],
                               [0, -1]], dtype=np.float32),
        "roberts_y": np.array([[0, 1],
                               [-1, 0]], dtype=np.float32),
        # Scharr Operators
        "scharr_x": np.array([[-3,  0,  3],
                              [-10, 0, 10],
                              [-3,  0,  3]], dtype=np.float32),
        "scharr_y": np.array([[-3, -10, -3],
                              [ 0,   0,  0],
                              [ 3,  10,  3]], dtype=np.float32),
    }
    
    features = {}
    total_energy = 0.0
    
    # Convolve the image with each kernel and compute the energy.
    for key, kernel in kernels.items():
        response = convolve(gray.astype(np.float32), kernel, mode="reflect")
        energy = np.sum(np.abs(response))
        features[f"edge_energy_2_{key}"] = energy
        total_energy += energy
    
    # Normalize the features by the total energy.
    if total_energy > 0:
        for key in features.keys():
            features[key] /= total_energy

    return features

In [81]:
def compute_glcm_descriptor(image):
    if image is None:
        return None
    
    if len(image.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    distance = [3]  
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4] 
    properties = ['contrast', 'correlation', 'energy', 'homogeneity']  
    
    glcm = graycomatrix(image, distances=distance, angles=angles, symmetric=True, normed=True)
    
    features = []
    for prop in properties:
        feature = graycoprops(glcm, prop).flatten()
        features.extend(feature)
    
    return np.array(features)

In [82]:
def create_gabor_filter(size, u0, v0, delta_x, delta_y):
    #size: kernel size
    #u0, v0: spatial frequency points
    #delta_x, delta_y: spatial scales

    y, x = np.mgrid[-size//2:size//2, -size//2:size//2]
    
    gaussian = np.exp(-0.5 * (x**2/delta_x**2 + y**2/delta_y**2))

    sinusoid = np.exp(-2j * np.pi * (u0*x + v0*y))
    
    return gaussian * sinusoid

def compute_gist_descriptor(image):

    if image is None:
        return None
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = image.astype(np.float32) / 255.0
    
    local_mean = cv2.GaussianBlur(image, (5, 5), 1.0)
    variance = cv2.GaussianBlur(image**2, (5, 5), 1.0) - local_mean**2
    variance[variance < 0] = 0 
    local_std = np.sqrt(variance)
    image = (image - local_mean) / (local_std + 1e-8)
    
    features = []
    scales = [2, 4, 8, 16]
    orientations = 8
    
    for delta in scales:
        for theta in range(orientations):
            angle = theta * np.pi / orientations
            u0 = np.cos(angle) / delta
            v0 = np.sin(angle) / delta
            
            gabor_filter = create_gabor_filter(size=31,u0=u0,v0=v0,delta_x=delta,delta_y=delta)
            
            filtered = cv2.filter2D(image, cv2.CV_32F, np.real(gabor_filter))
            
            block_h = image.shape[0] // 4
            block_w = image.shape[1] // 4
            
            for i in range(4):
                for j in range(4):
                    block = filtered[i*block_h:(i+1)*block_h, j*block_w:(j+1)*block_w]
                    energy = np.mean(np.abs(block))
                    features.append(energy)
    
    return np.array(features)


In [90]:
def extract_all_features(image):    
    features = {}
    #10
    lbp_hist = lbp_feature(image) 
    features.update(lbp_hist) 
    #4
    texture_features = texture_feature(image)
    features.update(texture_features) 
    #7
    hu_moments_features = hu_moments(image)
    features.update(hu_moments_features)
    #25
    zernike_moment_features = zernike_feature(image)
    features.update(zernike_moment_features)
    
    # color_structure_features = color_structure_descriptor(image)
    # features.update(color_structure_features)
    
    #12
    color_features = enhanced_color_feature(image)
    features.update(color_features) 
    #5
    edge_histogram_features = extract_edge_histogram_features(image)
    features.update(edge_histogram_features)
    
    #
    edge_histogram_features_2 = extract_edge_histogram_features_2(image)
    features.update(edge_histogram_features_2)
    
    # color_features = color_feature(image)
    # features.update(color_features) 
    #8
    shape_features = basic_feature(image)
    if shape_features is not None:
        features.update(shape_features)
    #16
    glcm_features = compute_glcm_descriptor(image)
    if glcm_features is not None:
        for i, val in enumerate(glcm_features):
            features[f'GLCM_{i}'] = val
    #512
    # gist_features = compute_gist_descriptor(image)
    # if gist_features is not None:
    #     for i, val in enumerate(gist_features):
    #         features[f'GIST_{i}'] = val

    return features


def process_directory(base_path):
    all_data = []
    image_paths = []

    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):
                if 'negative' in root:
                    label = 0
                else:
                    label = 1
                image_paths.append((os.path.join(root, file), label))
    for image_path, label in tqdm(image_paths, desc="Processing Images"):
        image = cv2.imread(image_path)
        if image is None:
            continue  

        features = extract_all_features(image)
        features["Label"] = label
 
        all_data.append(features)

    df = pd.DataFrame(all_data)

    return df

In [38]:

base_path = "/home/duyle/Rice_photos/BC-15"

In [91]:
base_path = r'/home/duyle/Rice_photos/TBR-45'
df = process_directory(base_path)

Processing Images:   0%|          | 0/2337 [00:00<?, ?it/s]/home/duyle/.local/lib/python3.13/site-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
Processing Images: 100%|██████████| 2337/2337 [02:12<00:00, 17.65it/s]


In [92]:
df

,LBP_0,LBP_1,LBP_2,LBP_3,LBP_4,LBP_5,LBP_6,LBP_7,LBP_8,LBP_9,...,GLCM_7,GLCM_8,GLCM_9,GLCM_10,GLCM_11,GLCM_12,GLCM_13,GLCM_14,GLCM_15,Label
0,504,1194,636,1956,3463,2999,1609,1300,2023,2176,...,0.876516,0.088325,0.080892,0.076210,0.078629,0.345646,0.285175,0.245843,0.269990,1
1,613,1207,699,1823,2903,2536,1342,1305,1727,2612,...,0.908837,0.084649,0.075170,0.071491,0.076803,0.313890,0.258703,0.232351,0.266753,1
2,380,882,499,1721,2976,2652,1317,998,1579,1582,...,0.932737,0.086506,0.076586,0.073120,0.078300,0.339207,0.265794,0.241247,0.284220,1
3,410,947,508,1804,2844,2587,1462,1103,1776,1808,...,0.935297,0.093560,0.082427,0.079417,0.085350,0.354369,0.283624,0.262094,0.303779,1
4,572,1058,746,1799,2827,2652,1413,1220,1672,1874,...,0.879619,0.077745,0.070611,0.067074,0.069385,0.313767,0.260577,0.235049,0.256235,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332,503,1042,604,1688,2910,2322,1143,1077,1473,1854,...,0.885183,0.068369,0.059321,0.054515,0.058399,0.295855,0.229553,0.192652,0.218205,0
2333,523,990,652,1775,2593,2699,1385,1186,1798,1876,...,0.897091,0.072664,0.065521,0.061782,0.065177,0.349146,0.294852,0.264624,0.293147,0
2334,773,1480,927,2370,3580,3209,1838,1661,2474,2688,...,0.883013,0.087901,0.080764,0.076535,0.080307,0.307358,0.258735,0.231358,0.251630,0
2335,411,1010,570,1893,3375,2999,1466,1188,1887,1745,...,0.941727,0.072607,0.065884,0.062700,0.065771,0.355383,0.302969,0.266115,0.299286,0


In [93]:
df.to_csv(f'all_with_zernike_pluscolor_enhanced_edge_edge2_TBR-45.csv',index=False)

In [84]:
types = ['BC-15','Huong_thom-1','Nep-87','Q-5','Thien_uu-8','Xi-23', 'TBR-36', 'TBR_45', 'TH3-5']
for type in types:
    df = process_directory(f'/home/duyle/Rice_photos/{type}')
    df.to_csv(f'all_with_zernike_pluscolor_enhanced_edge_{type}.csv',index=False)

Processing Images:   0%|          | 0/3422 [00:00<?, ?it/s]/home/duyle/.local/lib/python3.13/site-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
Processing Images: 100%|██████████| 2328/2328 [01:54<00:00, 20.30it/s]
Processing Images: 0it [00:00, ?it/s]
Processing Images:   0%|          | 0/2074 [00:00<?, ?it/s]/home/duyle/.local/lib/python3.13/site-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
Processing Images: 100%|██████████| 2074/2074 [01:48<00:00, 19.07it/s]


In [67]:
df

,LBP_0,LBP_1,LBP_2,LBP_3,LBP_4,LBP_5,LBP_6,LBP_7,LBP_8,LBP_9,...,GLCM_7,GLCM_8,GLCM_9,GLCM_10,GLCM_11,GLCM_12,GLCM_13,GLCM_14,GLCM_15,Label
0,249,667,355,1761,2894,2749,1194,921,1615,1329,...,0.919322,0.087261,0.076275,0.072825,0.076559,0.356618,0.279062,0.248866,0.286045,1
1,344,833,411,1739,2987,2735,1278,798,1460,1489,...,0.899277,0.077756,0.067537,0.063615,0.068074,0.340384,0.262537,0.228819,0.257738,1
2,485,1047,672,2118,3141,3007,1522,1202,2016,1966,...,0.910777,0.086017,0.078133,0.074423,0.078500,0.344613,0.284887,0.255701,0.285575,1
3,567,997,707,1824,2960,2570,1392,1184,1621,1873,...,0.903198,0.071537,0.063342,0.061175,0.064125,0.305813,0.246045,0.218532,0.250488,1
4,541,1004,560,1536,2739,2108,1078,1067,1325,2058,...,0.859189,0.064338,0.055301,0.052004,0.055064,0.291228,0.230163,0.199191,0.223099,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3417,549,1109,800,2156,3089,2942,1720,1276,2028,2061,...,0.917721,0.089944,0.083424,0.080165,0.083763,0.329930,0.285341,0.260048,0.288865,0
3418,325,909,524,2151,3341,3508,1509,1149,2382,2454,...,0.939424,0.110095,0.100542,0.097592,0.102480,0.380486,0.310400,0.287083,0.329649,0
3419,449,1067,592,2365,3376,3380,1773,1366,2238,2114,...,0.932853,0.077291,0.070693,0.067480,0.071971,0.376489,0.319709,0.289111,0.330268,0
3420,334,725,447,1605,3184,2582,1185,817,1292,1401,...,0.927336,0.070488,0.060200,0.055431,0.060148,0.334753,0.261893,0.232836,0.263993,0
